In [1]:
#https://medium.com/@viniciusqroz/using-keras-tuner-to-find-the-best-parameters-for-your-neural-network-model-2dc02e0a1203

In [2]:
!pip install keras-tuner

   ---------------------------------------- 129.1/129.1 kB 1.3 MB/s eta 0:00:00


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner.tuners import RandomSearch

Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow.random'

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28)))
    
    # Tune the number of units in the first dense layer
    # Choose an optimal value between 32-512
    hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units1, activation='relu'))
    
    # Tune the number of units in the second dense layer
    # Choose an optimal value between 32-512
    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    model.add(layers.Dense(units=hp_units2, activation='relu'))
    
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    hp_learning_rate = hp.Choice('learning_rate', values=[0.01, 0.001, 0.0001])
    
    model.add(layers.Dense(10, activation='softmax'))
    
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [ ]:
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='my_dir',
    project_name='helloworld')

In [ ]:
tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

In [ ]:
best_models = tuner.get_best_models(num_models=1)

In [ ]:
test_loss, test_acc = best_models[0].evaluate(x_test, y_test)
print('Test accuracy:', test_acc)